<center>数据处理与参数计算</center>

In [1]:
import pandas as pd 
import numpy as np 
import math

<center>处理十年期美债收益率,估计无风险收益率rf</center>

In [70]:
bond_data = pd.read_csv("十年期美债收益率241104-251104.csv")
bond_data['Date'] = pd.to_datetime(bond_data['observation_date'], format='%Y/%m/%d')
bond_data.set_index('Date', inplace=True)
bond_data.drop(columns=['observation_date'], inplace=True)
bond_data.head()
bond_data.isna().sum(axis=0)
missing_Data = bond_data[bond_data['DGS10'].isna()]
missing_Data
bond_data.sort_index(inplace=True)
monthly_bond_data = bond_data.resample('M').last()
monthly_bond_data
rf = monthly_bond_data['DGS10'].mean()*0.01
rf

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_32464\1638611291.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_bond_data = bond_data.resample('M').last()


np.float64(0.04276923076923076)

<center>计算腾讯股价的历史波动率volatility、股价收益率q</center>

In [71]:
#================数据处理==================
data = pd.read_csv(r"腾讯241105-251105日频数据.csv")
data = data.rename(columns={'Close/Last': 'Close'})
data.index = pd.to_datetime(data['Date'])
data.drop(columns=['Date'], inplace=True)
data.sort_index(axis=0,ascending=True,inplace=True)
data.shape
data['Close'].isna().sum(axis=0)
data['Close'] = data['Close'].str.replace('$','').astype(float)
print(data.head())
#================数据处理==================

#================计算历史波动率==================
data['log_return'] = np.log(data['Close'] / data['Close'].shift(1))
data.drop(axis=0,index=data.index[0],inplace=True)
print(data['log_return'].head())
data.shape
n = data.shape[0]
volatility = data['log_return'].std() * math.sqrt(n)
volatility
#================计算历史波动率==================

#================计算股价收益率==================
#近三年腾讯发放股息日期与金额
dividend_data = {
    '2023-05-18': [0.306,42.60],
    '2024-05-17': [0.434,51.08],
    '2025-05-19': [0.573,65.99]
}
dividend_data = pd.DataFrame.from_dict(dividend_data, orient='index', columns=['dividend_per_share', 'St'])
ri = dividend_data['dividend_per_share'] / dividend_data['St']
Ri = 1 + ri
ri,Ri
q = np.log(Ri.prod())/3
q
#================计算股价收益率==================
print("无风险收益率rf:", rf,"股票历史波动率volatility:", volatility, "股价收益率q:", q)

            Close   Volume     Open     High      Low
Date                                                 
2024-11-06  53.69  2526787  $53.44   $53.98   $53.05 
2024-11-07  56.07  2849317  $55.22   $56.35   $55.20 
2024-11-08  53.17  2592961  $53.91   $53.91   $52.86 
2024-11-11  52.50  2274080  $52.95   $52.95   $52.41 
2024-11-12  51.04  2982655  $51.80   $51.83   $50.79 
Date
2024-11-07    0.043374
2024-11-08   -0.053107
2024-11-11   -0.012681
2024-11-12   -0.028204
2024-11-13   -0.017990
Name: log_return, dtype: float64
无风险收益率rf: 0.04276923076923076 股票历史波动率volatility: 0.34472455189978907 股价收益率q: 0.008087886486253635
